In [ ]:
import matplotlib as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input,Dropout, MaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv('/content/alldata_1_for_kaggle.csv',encoding="latin-1")

In [ ]:
df.head()

,Unnamed: 0,0,a
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,4,Thyroid_Cancer,This study aimed to investigate serum matrix ...


In [ ]:
shape = np.shape(df)
print(shape)

(7570, 3)


In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.tail()

,0,a
7565,Colon_Cancer,we report the case of a 24yearold man who pres...
7566,Colon_Cancer,among synchronous colorectal cancers scrcs rep...
7567,Colon_Cancer,the heterogeneity of cancer cells is generally...
7568,Colon_Cancer,"""adipogenesis is the process through which mes..."
7569,Colon_Cancer,the periparturient period is one of the most c...


In [ ]:
df.rename({'0':'class_canc','a':'diagnostic'},axis=1, inplace=True)
df.head()

,class_canc,diagnostic
0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...
3,Thyroid_Cancer,Solitary plasmacytoma SP of the skull is an u...
4,Thyroid_Cancer,This study aimed to investigate serum matrix ...


In [ ]:
unique_attributes = df['class_canc'].unique()
print(unique_attributes)

['Thyroid_Cancer' 'Colon_Cancer' 'Lung_Cancer']


In [ ]:
class_counts = df['class_canc'].value_counts()
print(class_counts)

Thyroid_Cancer    2810
Colon_Cancer      2580
Lung_Cancer       2180
Name: class_canc, dtype: int64


In [ ]:
shape = np.shape(df)
print(shape)

(7570, 2)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7570 entries, 0 to 7569
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   class_canc  7570 non-null   object
 1   diagnostic  7570 non-null   object
dtypes: object(2)
memory usage: 118.4+ KB


In [ ]:
df['diagnostic'].dtype

dtype('O')

In [ ]:
Prétraitement

In [ ]:
X = df['diagnostic'].values
y = df['class_canc'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_seq_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_seq_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_seq_length)

In [ ]:
num_classes = len(label_encoder.classes_)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_seq_length))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
print(X_train_padded.dtype)
print(X_test_padded.dtype)

int32
int32


In [ ]:
print(y_train.dtype)
print(y_test.dtype)

object
object


In [ ]:
print(y_encoded.dtype)

int64


In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
print(y_train_encoded.dtype)
print(y_test_encoded.dtype)

int64
int64


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_padded, y_train_encoded, batch_size=32, epochs=10, validation_data=(X_test_padded, y_test_encoded))

Epoch 1/10
190/190 [==============================] - 92s 474ms/step - loss: 0.4414 - accuracy: 0.8415 - val_loss: 0.0421 - val_accuracy: 0.9855
Epoch 2/10
190/190 [==============================] - 89s 469ms/step - loss: 0.0376 - accuracy: 0.9858 - val_loss: 0.0281 - val_accuracy: 0.9901
Epoch 3/10
190/190 [==============================] - 89s 468ms/step - loss: 0.0275 - accuracy: 0.9894 - val_loss: 0.0190 - val_accuracy: 0.9947
Epoch 4/10
190/190 [==============================] - 89s 467ms/step - loss: 0.0225 - accuracy: 0.9922 - val_loss: 0.0229 - val_accuracy: 0.9914
Epoch 5/10
190/190 [==============================] - 89s 467ms/step - loss: 0.0177 - accuracy: 0.9932 - val_loss: 0.0169 - val_accuracy: 0.9927
Epoch 6/10
190/190 [==============================] - 89s 470ms/step - loss: 0.0169 - accuracy: 0.9937 - val_loss: 0.0130 - val_accuracy: 0.9921
Epoch 7/10
190/190 [==============================] - 89s 469ms/step - loss: 0.0167 - accuracy: 0.9934 - val_loss: 0.0298 - val_ac

In [ ]:
# Évaluez les performances du modèle sur les données de test
loss, accuracy = model.evaluate(X_test_padded, y_test_encoded)
print("Loss:", loss)
print("Accuracy:", accuracy)

48/48 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 0.9941
Loss: 0.009577670134603977
Accuracy: 0.9940555095672607
